<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spectrograms-to-TFRecord" data-toc-modified-id="Spectrograms-to-TFRecord-1">Spectrograms to TFRecord</a></span></li></ul></div>

In [59]:
METADATA_FILE = 'meta_data.csv'

TFR_FILE = 'spectorgrams.tfrecord'

In [60]:
import pandas as pd

meta_data = pd.read_csv(METADATA_FILE)
meta_data.sample(3)

,emotion,intensity,statement,repetition,actor,actor_name,file_name
1134,disgust,normal,2,1,female,actor_13,data/Actor_13/03-01-07-01-02-01-13.wav
1152,calm,strong,2,1,female,actor_09,data/Actor_09/03-01-02-02-02-01-09.wav
368,fearful,strong,1,2,male,actor_10,data/Actor_10/03-01-06-02-01-02-10.wav


In [ ]:
import tensorflow as tf

def create_text_feature(text):
    return tf.train.Feature(bytes_list=tf.train.BytesList(
        value=[text.encode('utf-8')]))


def crete_numpy_feature(array):
    array = tf.io.serialize_tensor(array).numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[array]))


def create_features(emotion, intensity, actor, mel_spect):
    return tf.train.Features(
        feature={
            'emotion': create_text_feature(emotion),
            'intensity': create_text_feature(intensity),
            'actor': create_text_feature(actor),
            'mel_spect': crete_numpy_feature(mel_spect)
        }
    )


### Spectrograms to TFRecord

In [58]:
from tqdm.notebook import tqdm
import librosa


with tf.io.TFRecordWriter(METADATA_FILE) as trf_writer:
    
    with tqdm(total=meta_data.shape[0]) as pbar:
        
        for item in meta_data.itertuples():

            y, sr = librosa.load(item.file_name)
            mel_spect = librosa.feature.melspectrogram(y=y, sr=sr)

            features = create_features(item.emotion, item.intensity, item.actor, mel_spect)
            example = tf.train.Example(features=features)

            trf_writer.write(example.SerializeToString())
            
            pbar.update(1)

KeyboardInterrupt: 